In [189]:
from __future__ import annotations

import pyvista
from pyvista import examples
from pathlib import Path

import vtk
import pyvista as pv
import pvxarray

import rioxarray as rxr
import numpy as np
import xarray as xr
import gemgis as gg



In [195]:
root_folder = Path(r"C:\Users\PBm\OneDrive - NGI\Shared\Leila")
scale = 5
folders = [folder for folder in root_folder.iterdir() if folder.is_dir()]

for folder in folders:

    if not (folder / f"pvista_{scale}m_texture.npy").exists():
        
        try:
            dem = rxr.open_rasterio(folder / "DEM.tif", masked=True)
            ortho = rxr.open_rasterio(folder / "orthomosaic.tif", masked=False)


            ortho_projected = ortho.rio.reproject_match(dem)
            ortho_projected = ortho_projected.assign_coords(x=dem.x, y=dem.y)

            dem_1m = dem.rio.reproject(dem.rio.crs, resolution=scale)
            ortho_projected = ortho.rio.reproject(dem.rio.crs, resolution=int(scale/2))
            ortho_projected = ortho_projected.astype(np.uint8)

            mesh = dem_1m.pyvista.mesh(x="x", y="y", z="band")
            o = ortho_projected.x.min(), ortho_projected.y.min(), 0
            u = ortho_projected.x.max(), ortho_projected.y.min(), 0
            v = ortho_projected.x.min(), ortho_projected.y.max(), 0
            mesh.texture_map_to_plane(o, u, v, inplace=True)
            mesh = mesh.warp_by_scalar()

            mesh.save(
                folder / f"pvista_{scale}m.vtk"
            )

            np.save(folder / f"pvista_{scale}m_texture.npy", ortho_projected.to_numpy().transpose(1, 2, 0)[:, :, :4])
        except Exception as e:
            print(f"Error processing {folder.name}: {e}")

        continue


Error processing Laviner: 'C:\Users\PBm\OneDrive - NGI\Shared\Leila\Laviner\orthomosaic.tif' not recognized as being in a supported file format.


In [ ]:
folders

In [180]:
# Load the mesh file
folder = Path(r"C:\Users\PBm\OneDrive - NGI\Shared\Leila\2. Surgonda et Corn margun")
loaded_mesh = pv.read(folder / f"pvista_{scale}m.vtk")

# Load the texture data from the .npy file
loaded_texture = np.load(folder / f"pvista_{scale}m_texture.npy")

In [181]:
pl = pyvista.Plotter(notebook=True, lighting='three lights')
pl.camera.zoom(1.2)


#pl.iren.add_observer(vtk.vtkCommand.EndInteractionEvent, my_cpos_callback)
pl.add_mesh(loaded_mesh, opacity=1, texture=loaded_texture)
pl.show()

Widget(value='<iframe src="http://localhost:64926/index.html?ui=P_0x217bd0a5310_41&reconnect=auto" class="pyvi…

In [ ]:
pv.save_meshio

AttributeError: 'StructuredGrid' object has no attribute 'textures'

In [ ]:
mesh = ortho_projected.pyvista.mesh(x="x", y="y", component="band")

c:\Users\PBm\AppData\Local\miniconda3\envs\leila\Lib\site-packages\pvxarray\rectilinear.py:41: DataCopyWarning: Made a copy of the multicomponent array - VTK/PyVista data not shared with xarray.
  warnings.warn(
